# Intsall Package

In [5]:
# Install relevant packages, if not yet
!pip3 install awswrangler
!pip3 install pandasql
!pip3 install -U nltk
!pip3 install stop-words
!pip3 install gensim


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
  Using cached stop_words-2018.7.23-py3-none-any.whl

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
  Using cached gensim-4.2.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from data_connector import *
import numpy as np
import pandas as pd

# Import Packages and Connect to RedShift

In [2]:
# Fill in the relevant credentials for Redshift cluster(s) and S3 bucket(s) to use
role_arn = 'arn:aws:iam::848271137894:role/RedShiftAccessFromSageMakerAccount'
role_session_name = 'AssumeRoleSession1'
secret_name = 'arn:aws:secretsmanager:us-west-2:848271137894:secret:Redshift-DLL-Sagemaker-Secret-nHWV37'
dbname = 'ucicanvas'
region_name = 'us-west-2'
bucket_name = 'uci-dll-mellon-redshift-unload-us-west-2'
unload_iam_role = 'arn:aws:iam::848271137894:role/uci-write-to-dll-mellon-redshift-unload-s3-bucket-role,arn:aws:iam::686457044982:role/uci-oit-dwh-redshift-to-mellon-s3-unload-role'

In [3]:
# Initiate a connector object
redshift_s3_connect = RedshiftS3Connector(role_arn, role_session_name, secret_name, dbname, bucket_name, region_name, unload_iam_role)

# 1. Get the mellon_layer2.discussion_post table from Fall 2020 to Spring 2022

In [6]:
query = f'''SELECT * FROM mellon_layer2.student LIMIT 1000;'''
df = redshift_s3_connect.run_sql_command(query)

Done waiting to finish Data API.
Status: FINISHED. Excution time: 358 miliseconds
{'ClusterIdentifier': 'uci-datalake-canvas-cluster-v1-redshiftcluster-a9h29wndif8j', 'CreatedAt': datetime.datetime(2022, 7, 27, 23, 30, 13, 506000, tzinfo=tzlocal()), 'Duration': 358856185, 'HasResultSet': True, 'Id': '61c59936-7ae7-45f3-87ba-3e808eebe345', 'QueryString': 'SELECT * FROM mellon_layer2.student LIMIT 1000;', 'RedshiftPid': 1078545018, 'RedshiftQueryId': 4775086, 'ResultRows': 1000, 'ResultSize': 422970, 'SecretArn': 'arn:aws:secretsmanager:us-west-2:848271137894:secret:Redshift-DLL-Sagemaker-Secret-nHWV37', 'Status': 'FINISHED', 'UpdatedAt': datetime.datetime(2022, 7, 27, 23, 30, 14, 397000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'e56278e9-11ac-4390-90af-b1e13cad3489', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e56278e9-11ac-4390-90af-b1e13cad3489', 'content-type': 'application/x-amz-json-1.1', 'content-length': '503', 'date': 'Wed, 27 Jul 2022 23:30:33 GMT'},

In [10]:
df['geo_category'].unique()

array(['Other CA', 'Non-OC Southern CA', 'Orange County',
       'Foreign Student', 'Out of State US'], dtype=object)

In [21]:
df['geo_category'].unique()

array(['Other CA', 'Non-OC Southern CA', 'Orange County',
       'Foreign Student', 'Out of State US'], dtype=object)

In [11]:
get_df_query = f'''
WITH roster AS
         (SELECT DISTINCT mellon_id,
                          sis_term_code,
                          instruc_begins::DATE,
                          (instruc_ends::DATE + 9) AS final_ends,
                          canvas_course_id,
                          sis_course_code,
                          canvas_course_section_id,
                          sis_course_section_code
          FROM mellon_layer2.enrollment er
                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc
          WHERE is_valid_user
            AND is_valid_course_section
            AND enrollment_type = \\'StudentEnrollment\\')
SELECT dp.*, rs.sis_term_code AS term, st.citizenship_app AS citizenship, st.geo_category
FROM mellon_layer2.v_discussion_post AS dp 
INNER JOIN 
(SELECT DISTINCT mellon_id, canvas_course_id, instruc_begins, final_ends, sis_term_code FROM roster) AS rs
                              ON dp.canvas_course_id = rs.canvas_course_id AND dp.mellon_id = rs.mellon_id
LEFT JOIN mellon_layer2.student st 
                              ON dp.mellon_id = st.mellon_id
WHERE CONVERT_TIMEZONE(\\'PST\\', dp.discussion_post_created_at) BETWEEN \\'2020-10-01 00:00:00\\' AND \\'2022-06-10 00:00:00\\' 
        AND CONVERT_TIMEZONE(\\'PST\\', dp.discussion_post_created_at) BETWEEN rs.instruc_begins AND rs.final_ends
        AND dp.discussion_post_deleted_at IS NULL;'''
print(get_df_query)


WITH roster AS
         (SELECT DISTINCT mellon_id,
                          sis_term_code,
                          instruc_begins::DATE,
                          (instruc_ends::DATE + 9) AS final_ends,
                          canvas_course_id,
                          sis_course_code,
                          canvas_course_section_id,
                          sis_course_section_code
          FROM mellon_layer2.enrollment er
                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc
          WHERE is_valid_user
            AND is_valid_course_section
            AND enrollment_type = \'StudentEnrollment\')
SELECT dp.*, rs.sis_term_code AS term, st.citizenship_app AS citizenship, st.geo_category
FROM mellon_layer2.v_discussion_post AS dp 
INNER JOIN 
(SELECT DISTINCT mellon_id, canvas_course_id, instruc_begins, final_ends, sis_term_code FROM roster) AS rs
                              ON dp.canvas_course_id = rs.canvas_course_id AND dp.mellon

In [2]:
file_in_s3 = 'discussion_post_2020_2022.csv'

In [13]:
redshift_s3_connect.unload_from_redshift_to_s3(get_df_query, file_in_s3, 'csv')

Waiter DataAPIExecution failed: Max attempts exceeded
Status: STARTED. Excution time: 0 miliseconds
{'ClusterIdentifier': 'uci-datalake-canvas-cluster-v1-redshiftcluster-a9h29wndif8j', 'CreatedAt': datetime.datetime(2022, 7, 27, 23, 41, 24, 193000, tzinfo=tzlocal()), 'Duration': -1, 'HasResultSet': False, 'Id': '8d011573-f9ef-4b9c-8d2e-b6b1c222502f', 'QueryString': "UNLOAD ('\nWITH roster AS\n         (SELECT DISTINCT mellon_id,\n                          sis_term_code,\n                          instruc_begins::DATE,\n                          (instruc_ends::DATE + 9) AS final_ends,\n                          canvas_course_id,\n                          sis_course_code,\n                          canvas_course_section_id,\n                          sis_course_section_code\n          FROM mellon_layer2.enrollment er\n                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc\n          WHERE is_valid_user\n            AND is_valid_course_section\n      

In [9]:
file_names = [file_in_s3]
file_type = 'csv'
discussion_post = redshift_s3_connect.read_from_s3(file_names, file_type)
discussion_post.head(5)

,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,parent_discussion_post_id,discussion_topic_id,...,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term,citizenship,geo_category
0,3110931,172865,30691,<p>I used to simply read through my papers and...,2020-11-02 07:49:30.285,2020-11-02 07:49:30.285,NaN,1,NaN,403139,...,NaN,2020-09-27 19:56:25.349,2020-09-30 23:53:36.699,NaN,f,NaN,591903.0,Fall 2020,Permanent Resident,Non-OC Southern CA
1,3110964,172865,30691,"<p>Hello Ivan,</p>\n<p>I've never thought abou...",2020-11-02 07:51:33.184,2020-11-02 07:51:33.184,NaN,2,3107124.0,403139,...,NaN,2020-09-27 19:56:25.349,2020-09-30 23:53:36.699,NaN,f,NaN,591903.0,Fall 2020,Permanent Resident,Non-OC Southern CA
2,3061307,172865,30691,"<p>Using mobile devices in a business setting,...",2020-10-26 06:50:15.231,2020-10-26 06:50:15.231,NaN,1,NaN,403138,...,NaN,2020-09-27 19:56:22.80,2020-09-30 23:53:17.864,NaN,f,NaN,591902.0,Fall 2020,Permanent Resident,Non-OC Southern CA
3,3202024,172865,30691,<p>Credibility is an important point of persua...,2020-11-16 07:52:04.144,2020-11-16 07:52:04.144,NaN,1,NaN,403141,...,NaN,2020-09-27 20:09:55.933,2020-09-30 23:53:58.101,NaN,f,NaN,591907.0,Fall 2020,Permanent Resident,Non-OC Southern CA
4,3202081,172865,30691,"<p>Hi Allya,</p>\n<p>I didn't even consider th...",2020-11-16 07:54:01.969,2020-11-16 07:54:01.969,NaN,2,3201255.0,403141,...,NaN,2020-09-27 20:09:55.933,2020-09-30 23:53:58.101,NaN,f,NaN,591907.0,Fall 2020,Permanent Resident,Non-OC Southern CA


In [10]:
len(discussion_post)

1168344

In [11]:
discussion_post.to_csv('discussion_post_2020_2022.csv')

In [4]:
df = pd.read_csv('discussion_post_2020_2022.csv')

In [5]:
df.head(3)

,Unnamed: 0,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,parent_discussion_post_id,...,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term,citizenship,geo_category
0,0,3110931,172865,30691,<p>I used to simply read through my papers and...,2020-11-02 07:49:30.285,2020-11-02 07:49:30.285,NaN,1,NaN,...,NaN,2020-09-27 19:56:25.349,2020-09-30 23:53:36.699,NaN,f,NaN,591903.0,Fall 2020,Permanent Resident,Non-OC Southern CA
1,1,3110964,172865,30691,"<p>Hello Ivan,</p>\n<p>I've never thought abou...",2020-11-02 07:51:33.184,2020-11-02 07:51:33.184,NaN,2,3107124.0,...,NaN,2020-09-27 19:56:25.349,2020-09-30 23:53:36.699,NaN,f,NaN,591903.0,Fall 2020,Permanent Resident,Non-OC Southern CA
2,2,3061307,172865,30691,"<p>Using mobile devices in a business setting,...",2020-10-26 06:50:15.231,2020-10-26 06:50:15.231,NaN,1,NaN,...,NaN,2020-09-27 19:56:22.80,2020-09-30 23:53:17.864,NaN,f,NaN,591902.0,Fall 2020,Permanent Resident,Non-OC Southern CA


In [17]:
df.groupby(by='geo_category')['mellon_id'].nunique().reset_index()

,geo_category,mellon_id
0,Foreign Student,2106
1,Non-OC Southern CA,9293
2,Orange County,4803
3,Other CA,3821
4,Out of State US,606


In [12]:
df_small = discussion_post.sample(n=100000)

In [13]:
df_small.to_csv('discussion_post_2020_2022_small.csv')

In [14]:
discussion_post.groupby(by='term')['discussion_post_id'].count().reset_index(name='discussion_post_cnt').reindex([0, 4, 2, 1, 5, 3])

,term,discussion_post_cnt
0,Fall 2020,300551
4,Winter 2021,268359
2,Spring 2021,250647
1,Fall 2021,140963
5,Winter 2022,116430
3,Spring 2022,91394


# 2. Preprocess the data

## 2.1 Clean the data

In [2]:
import online_discourse_analysis as oda
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('discussion_post_2020_2022.csv', index_col=False)

In [ ]:
df = oda.discussion_text_clean(df)

/root/Discussion_Behavior_Project/online_discourse_analysis.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cleaned_discussion_post'] = df['discussion_post_content'].str.replace(regex, '')


In [5]:
df

1168344

## 2.2 Transfer the UTC timestamp to Pacific timestamp

In [14]:
df = oda.utc_to_pts(df_small)

In [10]:
df.head(5)

,Unnamed: 0,Unnamed: 0.1,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,...,discussion_topic_id,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term
0,818696,818696,3710696,184024,33621,<p>Going through the Kriger et. al reading sol...,2021-02-22 02:21:08.093000+00:00,2021-02-22 02:21:08.093,NaN,1,...,465175,Week 7 RPR,NaN,2020-12-28 17:13:43.494,2021-02-15 04:25:16.831000+00:00,NaN,f,NaN,654138.0,Winter 2021
1,1163402,1163402,3280790,189732,29719,<p>The drug I am looking for is loratadine. Th...,2020-11-30 06:14:56.718000+00:00,2020-11-30 06:14:56.718,NaN,1,...,408802,Week 8 Discussion 1 (Side-effects) - Group 27,NaN,2020-10-01 01:43:16.391,2020-10-01 01:43:16.391000+00:00,NaN,f,98950.0,575833.0,Fall 2020
2,1069337,1069337,3937321,180683,35411,<p>I totally agree with the idea that stress w...,2021-04-12 00:23:58.330000+00:00,2021-04-12 00:23:58.33,NaN,1,...,521980,"Discussion 2: The Etiology, Physiology, Sympto...",NaN,2021-03-20 00:55:55.57,2021-03-20 00:55:55.570000+00:00,NaN,f,122281.0,709312.0,Spring 2021
3,279733,279733,3419760,173435,34214,<p>Hello everyone!! My name is Darshit Shethna...,2021-01-09 10:59:23.040000+00:00,2021-01-09 10:59:54.513,NaN,1,...,478022,INTRODUCTIONS,NaN,2021-01-06 07:09:05.054,2021-01-06 07:09:05.469000+00:00,NaN,f,NaN,675548.0,Winter 2021
4,1055184,1055184,3812426,196511,34162,<p><span>Hypoactive Sexual Desire Disorder (AK...,2021-03-11 06:52:05.860000+00:00,2021-03-11 06:52:05.86,NaN,1,...,512474,Discussion 10: Health & Medicine - Group 4,NaN,2021-03-08 03:16:04.695,2021-03-08 03:16:04.698000+00:00,NaN,f,110753.0,702805.0,Winter 2021


# 3. Calculate Behavioral Features

In [15]:
df = oda.discussion_topic_post_time_gap(df)

In [17]:
df = oda.calculate_features_avg(df, ['discussion_topic_post_time_gap'])

In [8]:
df_cnt = oda.discussion_post_count(df)

In [ ]:
df = df.merge(df_cnt, how='inner', on='term')

In [18]:
df.head(10)

,term,discussion_topic_post_time_gap_avg
0,0,30.423914
1,Fall 2020,21.495316
2,Fall 2021,27.240580
3,Spring 2021,19.848608
4,Spring 2022,27.877707
5,Summer 2021,17.694850
6,Winter 2021,17.389650
7,Winter 2022,19.990592


# Calculate Reply Rates

In [9]:
df = pd.read_csv('discussion_post_2020_2022.csv')

In [10]:
df = oda.discussion_reply_rate(df)

In [11]:
df1 = df[['discussion_post_id', 'discussion_post_content', 'reply_rate']]

In [12]:
df1 = oda.discussion_text_clean(df1)

/root/Discussion_Behavior_Project/online_discourse_analysis.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_discussion_post'] = df['discussion_post_content'].str.replace(regex, '')


# LDA Models

## High Reply Rate Analysis

In [6]:
!pip3 install -U nltk
!pip3 install stop-words
!pip3 install gensim

  Using cached stop_words-2018.7.23-py3-none-any.whl
  Using cached gensim-4.2.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)


In [75]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [76]:
tokenizer = RegexpTokenizer(r"[\w']+")

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [49]:
df_hr = df1[df1['reply_rate'] > 2]

In [50]:
df_hr.shape

(23, 4)

In [51]:
df_hr.head(5)

,discussion_post_id,discussion_post_content,reply_rate,cleaned_discussion_post
81,3545388,<p><span>PROMPT 4 You are invited to initiate...,5,PROMPT 4 You are invited to initiate your own...
162,3993887,<p>Maslow's hierarchy generally fits with my e...,3,Maslow's hierarchy generally fits with my expe...
276,3863600,"<p><iframe style=""width: 400px; height: 225px;...",4,;;
372,4232164,"<p>Interestingly enough, social disputes can b...",3,"Interestingly enough, social disputes can be s..."
578,4041433,"<p>When I think about social media platforms, ...",3,"When I think about social media platforms, the..."


In [16]:
for i in range(0, 5):
    print(df_hr['cleaned_discussion_post'].iloc[i])
    print()

PROMPT 4  You are invited to initiate your own discussion by selecting a portion of an assigned text to comment on that wasn't addressed in the prompts above.  This discussion may include asking questions about what an author meant, disputing their conclusions, or explaining what their ideas mean to you on a personal level. Culture War/James Hunter/Page 225In his essay, James Hunter says that "the mass media act act as a filter through which our perceptions of the world around us take shape" (Hunter 225). He believes that the mass media has the power to change and reshape people' mind, perception and spiritual value. And as he claims in the following essay, those who own the mass media would also hold enormous power at the same time. In many cases, the groups hold the power of mass media are more conservative because they want to keep their value on the main stage instead of being taken place by something new. In that case, when there is a new culture value emerging, the conservatives 

In [52]:
# compile the cleaned_discussion_post column into a list
doc_set = [str(x) for x in list(df_hr['cleaned_discussion_post'].values)]
print(len(doc_set))

23


In [77]:
# list for tokenized documents in loop
texts = []

In [78]:
# loop through document list
for content in doc_set:
    
    # clean and tokenize document string
    raw = content.lower()
    tokens = tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [w for w in tokens if w not in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

In [79]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [80]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=1)

In [81]:
print(ldamodel.print_topics(num_topics=5, num_words=4))

[(0, '0.007*"movi" + 0.006*"class" + 0.005*"use" + 0.005*"year"'), (1, '0.008*"read" + 0.008*"time" + 0.006*"just" + 0.006*"women"'), (2, '0.011*"mass" + 0.010*"group" + 0.009*"power" + 0.009*"media"'), (3, '0.012*"group" + 0.010*"make" + 0.007*"person" + 0.006*"use"'), (4, '0.011*"senior" + 0.010*"http" + 0.009*"stress" + 0.009*"retriev"')]


In [82]:
for i in range(0, 4):
    print(texts[i])
    print()

['prompt', '4', 'invit', 'initi', 'discuss', 'select', 'portion', 'assign', 'text', 'comment', 'address', 'prompt', 'discuss', 'may', 'includ', 'ask', 'question', 'author', 'meant', 'disput', 'conclus', 'explain', 'idea', 'mean', 'person', 'level', 'cultur', 'war', 'jame', 'hunter', 'page', '225in', 'essay', 'jame', 'hunter', 'say', 'mass', 'media', 'act', 'act', 'filter', 'percept', 'world', 'around', 'us', 'take', 'shape', 'hunter', '225', 'believ', 'mass', 'media', 'power', 'chang', 'reshap', "people'", 'mind', 'percept', 'spiritu', 'valu', 'claim', 'follow', 'essay', 'mass', 'media', 'also', 'hold', 'enorm', 'power', 'time', 'mani', 'case', 'group', 'hold', 'power', 'mass', 'media', 'conserv', 'want', 'keep', 'valu', 'main', 'stage', 'instead', 'taken', 'place', 'someth', 'new', 'case', 'new', 'cultur', 'valu', 'emerg', 'conserv', 'alway', 'wield', 'mass', 'media', 'attack', 'new', 'cultur', 'valu', 'order', 'secur', 'will', 'caus', 'cultur', 'conflict', 'upgrad', 'cultur', 'war', 

# Remove running query

In [ ]:
redshift_s3_connect.view_running_sql_commands()

In [ ]:
pid = 'pid'
redshift_s3_connect.cancel_sql_command(pid)